# AutoCron Demo 5: Safe Mode

This notebook demonstrates AutoCron's safe mode for secure task execution with resource limits.

## Features Covered:
- Subprocess isolation
- Memory limits
- CPU limits (Unix/Linux/macOS)
- Timeout enforcement
- Output sanitization
- Running untrusted scripts safely

## Setup

In [ ]:
from autocron import AutoCron
from datetime import datetime
import sys
import platform

In [ ]:
# Check platform capabilities
os_type = platform.system()
print(f"️ Operating System: {os_type}")
print(f" Python: {sys.version.split()[0]}\n")

if os_type == "Windows":
 print("️ Windows: Subprocess isolation + timeout supported")
 print(" Windows: Memory/CPU limits not yet supported")
else:
 print(" Unix/Linux/macOS: Full safe mode support")
 print(" Subprocess isolation + memory + CPU limits + timeout")

## 1. Basic Safe Mode

Run tasks in subprocess isolation for safety.

In [ ]:
test_script = '''#!/usr/bin/env python3
import time
print("Starting safe task...")
time.sleep(2)
print("Task completed successfully!")
'''

# Save script to file
with open("safe_task.py", "w") as f:
 f.write(test_script)

print(" Created test script: safe_task.py")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="safe_script",
 script="safe_task.py",
 every='30s',
 safe_mode=True # Enable subprocess isolation
)

print(" Task scheduled with safe mode enabled")
print(" Script runs in isolated subprocess")
print(" Parent process is protected from script failures")

## 2. Memory Limits (Unix/Linux/macOS)

Prevent tasks from consuming too much memory.

In [ ]:
memory_script = '''#!/usr/bin/env python3
import sys
print("Allocating memory...")

try:
 # Try to allocate 512MB
 data = bytearray(512 * 1024 * 1024)
 print(f"Allocated {len(data) / 1024 / 1024:.0f} MB")
except MemoryError:
 print(" MemoryError: Hit memory limit!")
 sys.exit(1)
'''

with open("memory_task.py", "w") as f:
 f.write(memory_script)

print(" Created memory-hungry script")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="memory_limited",
 script="memory_task.py",
 every='1m',
 safe_mode=True,
 max_memory_mb=256 # Limit to 256MB
)

if os_type != "Windows":
 print(" Task scheduled with 256MB memory limit")
 print(" Script tries to allocate 512MB but limited to 256MB")
 print("️ Will be terminated if it exceeds the limit")
else:
 print("️ Memory limits not supported on Windows yet")

## 3. CPU Limits (Unix/Linux/macOS)

Prevent tasks from monopolizing CPU resources.

In [ ]:
cpu_script = '''#!/usr/bin/env python3
import time
print("Starting CPU-intensive task...")

start = time.time()
iterations = 0

# Busy loop for 10 seconds
while time.time() - start < 10:
 iterations += 1
 _ = sum(range(10000))

print(f"Completed {iterations:,} iterations")
'''

with open("cpu_task.py", "w") as f:
 f.write(cpu_script)

print(" Created CPU-intensive script")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="cpu_limited",
 script="cpu_task.py",
 every='2m',
 safe_mode=True,
 max_cpu_percent=50.0 # Limit to 50% CPU
)

if os_type != "Windows":
 print(" Task scheduled with 50% CPU limit")
 print(" Script CPU usage will be throttled to 50%")
 print("️ Prevents system lockup from runaway tasks")
else:
 print("️ CPU limits not supported on Windows yet")

## 4. Timeout Enforcement

Terminate tasks that run too long (works on all platforms).

In [ ]:
slow_script = '''#!/usr/bin/env python3
import time
print("Starting long task...")

for i in range(1, 16):
 print(f"Working... {i}/15 seconds")
 time.sleep(1)

print("Task completed!")
'''

with open("slow_task.py", "w") as f:
 f.write(slow_script)

print(" Created slow script (takes 15 seconds)")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="timeout_task",
 script="slow_task.py",
 every='1m',
 safe_mode=True,
 timeout=5 # Kill after 5 seconds
)

print(" Task scheduled with 5-second timeout")
print(" Script takes 15 seconds but will be killed at 5 seconds")
print("⏱️ Timeout works on all platforms (Windows, Linux, macOS)")

## 5. Complete Safe Mode Configuration

Combine all safety features for maximum protection.

In [ ]:
untrusted_script = '''#!/usr/bin/env python3
import time
import sys

print("User script starting...")

# Simulate work
for i in range(10):
 print(f"Processing {i+1}/10...")
 time.sleep(0.5)
 
 # Try to allocate memory
 data = bytearray(10 * 1024 * 1024) # 10MB per iteration

print("Script completed successfully!")
'''

with open("untrusted_task.py", "w") as f:
 f.write(untrusted_script)

print(" Created untrusted script")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="untrusted_user_script",
 script="untrusted_task.py",
 every='5m',
 safe_mode=True, # Subprocess isolation
 max_memory_mb=128, # 128MB memory limit
 max_cpu_percent=25.0, # 25% CPU limit
 timeout=10 # 10 second timeout
)

print(" Untrusted script scheduled with complete protection:\n")
print(" Subprocess isolation: ON")
print(" Memory limit: 128 MB")
print(f" ️ CPU limit: 25% {'(Unix only)' if os_type == 'Windows' else ''}")
print(" ⏱️ Timeout: 10 seconds")
print("\n Script is fully sandboxed and cannot harm the system!")

## 6. Output Sanitization

Safe mode automatically limits output size to prevent DoS.

In [ ]:
noisy_script = '''#!/usr/bin/env python3
# Try to flood output
for i in range(10000):
 print(f"Line {i}: {'x' * 100}")
print("Done!")
'''

with open("noisy_task.py", "w") as f:
 f.write(noisy_script)

print(" Created noisy script (tries to generate lots of output)")

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
 name="noisy_task",
 script="noisy_task.py",
 every='5m',
 safe_mode=True
)

print(" Noisy task scheduled with safe mode")
print(" Output automatically limited to 10KB")
print("️ Prevents output flooding DoS attacks")

## 7. Real-World Example: Multi-Tenant System

Run user-provided scripts safely in a multi-tenant environment.

In [ ]:
class SafeTaskRunner:
 """Safe execution of user-provided tasks."""
 
 def __init__(self):
 self.scheduler = AutoCron()
 self.is_unix = platform.system() != "Windows"
 
 def add_user_task(self, user_id, script_path, schedule):
 """Add a user's task with appropriate safety limits."""
 task_name = f"user_{user_id}_task"

 # Base configuration (works on all platforms)
 config = {
 "name": task_name,
 "script": script_path,
 "every": schedule,
 "safe_mode": True,
 "timeout": 300, # 5 minutes max
 }

 if self.is_unix:
 config |= {
 "max_memory_mb": 512, # 512MB per user task
 "max_cpu_percent": 50.0, # 50% CPU max
 }

 self.scheduler.add_task(**config)

 print(f" Added task for user {user_id}")
 print(f" Script: {script_path}")
 print(f" Schedule: {schedule}")
 print(f" Safety: {'Full (Unix)' if self.is_unix else 'Subprocess + Timeout (Windows)'}\n")
 
 def start(self):
 print(" Starting safe task runner...")
 print(f" Running {len(self.scheduler.tasks)} user tasks safely\n")
 # self.scheduler.start() # Uncomment to actually start

runner = SafeTaskRunner()

# Simulate multiple users submitting scripts
runner.add_user_task(user_id=101, script_path="user_101_script.py", schedule="1h")
runner.add_user_task(user_id=102, script_path="user_102_script.py", schedule="30m")
runner.add_user_task(user_id=103, script_path="user_103_script.py", schedule="15m")

print(" All user scripts run in isolated sandboxes!")
print("️ No user can affect other users or the system")

## 8. Security Best Practices

In [ ]:
print(" Safe Mode Security Best Practices\n")
print("="*60)

print("\n1. ALWAYS use safe_mode for untrusted scripts")
print(" - User-provided code")
print(" - Third-party scripts")
print(" - Scripts from external sources")

print("\n2. ⏱️ ALWAYS set timeouts")
print(" - Prevents infinite loops")
print(" - Guarantees task completion")
print(" - Works on all platforms")

print("\n3. SET memory limits (Unix/Linux/macOS)")
print(" - Prevents OOM crashes")
print(" - Protects system stability")
print(" - Enables fair resource sharing")

print("\n4. ️ SET CPU limits (Unix/Linux/macOS)")
print(" - Prevents system lockup")
print(" - Fair CPU sharing")
print(" - Maintains system responsiveness")

print("\n5. MONITOR task execution")
print(" - Check for repeated failures")
print(" - Watch resource usage")
print(" - Review output logs")

print("\n6. VALIDATE scripts before scheduling")
print(" - Code review for untrusted sources")
print(" - Static analysis")
print(" - Test in isolated environment first")

print("\n" + "="*60)

## Cleanup

Remove demo scripts created in this notebook.

In [ ]:
import os

demo_scripts = [
 "safe_task.py",
 "memory_task.py",
 "cpu_task.py",
 "slow_task.py",
 "untrusted_task.py",
 "noisy_task.py"
]

for script in demo_scripts:
 if os.path.exists(script):
 os.remove(script)
 print(f"️ Removed: {script}")

print("\n Cleanup complete!")

## Summary

In this demo, you learned:

 **Subprocess Isolation** - Tasks run in separate processes (all platforms)

 **Memory Limits** - Prevent OOM crashes (Unix/Linux/macOS)

 **CPU Limits** - Prevent system lockup (Unix/Linux/macOS)

 **Timeout Enforcement** - Kill runaway tasks (all platforms)

 **Output Sanitization** - Prevent output flooding (all platforms)

 **Multi-Tenant Safety** - Run user scripts securely

### Platform Support:

**Windows:**
- Subprocess isolation
- Timeout enforcement
- ⏳ Memory/CPU limits coming soon

**Unix/Linux/macOS:**
- Full safe mode support
- All resource limits available

### Use Cases:

- **SaaS Platforms** - User-provided automation scripts
- **Educational** - Student code execution
- **CI/CD** - Running untrusted build scripts
- **Web Services** - Processing user uploads
- **Multi-Tenant** - Isolated task execution per customer

### Next Steps:
- Check out `06_dashboard.ipynb` for visual monitoring
- See `07_notifications.ipynb` for alerts and notifications
- Explore `08_cli.ipynb` for command-line interface